# Import

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json
import pickle
from bs4 import BeautifulSoup
import os
from tqdm.notebook import tqdm
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout
from json import JSONDecodeError
from pandas import json_normalize
import concurrent.futures


pd.set_option('display.max_columns', None)

# Ingestion

In [2]:
#Iterate all of the dictionary
path = 'replace with your path that contains the json/pickle files that have a list of topic id and key(keyword)'

response_dict_list = []

for filename in os.listdir(path):
    if filename.endswith('.pkl'):
        with open(f'{path}/{filename}', 'rb') as f:
            response_dict = pickle.load(f)
            response_dict_list.append(response_dict)
            
print(len(response_dict_list))

8


In [6]:
# Initialize an empty dictionary to store results
extracted_ids_keyword_dict = {}

def update_extracted_ids(response_dict_list, extracted_ids_keyword_dict):
    for response_dict in response_dict_list:
        for key, value in response_dict.items():
            if isinstance(value,dict) and 'search_keyword' in value.keys():
                search_keyword = value['search_keyword']  # Ensure 'search_keyword' exists
                if search_keyword:
                    if search_keyword not in extracted_ids_keyword_dict:
                        extracted_ids_keyword_dict[search_keyword] = []

                    for k, v in value.items():
                        if isinstance(v, dict) and 'data' in v.keys():
                            for data_item in v['data']:
                                extracted_ids_keyword_dict[search_keyword].append(data_item['id'])

    return extracted_ids_keyword_dict

# Update the dictionary with data from response_dict_list
extracted_ids_keyword_dict = update_extracted_ids(response_dict_list, extracted_ids_keyword_dict)

In [7]:
len(extracted_ids_keyword_dict.keys())

47

In [12]:
extracted_ids_keyword_dict.keys()

dict_keys(['บำนาญ', 'หุ้นปันผล', 'หุ้นต่างประเทศ', 'อสังหา', 'หุ้นเน้นปันผล ', 'กองทุน', 'สินทรัพย์', 'วางแผนชีวิต', 'วางแผนครอบครัว', 'แผนดูแลพ่อแม่', 'Work-Life Balance', 'เก็บเงินแต่งงาน', 'ให้เงินภรรยา', 'ส่งเงินให้ที่บ้าน', 'ดูแลพ่อแม่', 'เสียสละเพื่อครอบครัว', 'ทดแทนบุญคุณ', 'พ่อแม่ป่วย', 'ประกัน', 'ข้อมูลสุขภาพประกัน', 'เดินตามรอย', 'heart rate', 'ข้อมูลสุขภาพ', 'Passive Income', 'กระแสเงินสด', 'เป้าหมายการเงิน', 'รายได้เสริม', 'อาชีพเสริม', 'รายได้2ทาง', 'อิสรภาพทางการเงิน', 'รายได้หลายทาง', 'แยกกระเป๋า', 'กลยุทธ์การลงทุน', 'แนวทางการลงทุน', 'กลยุทธ์การเงิน', 'Early retire', 'เกษียณก่อนวัย', 'เก็บเงินไว้ใช้ตอนเกษียณ', 'เงินเก็บฉุกเฉิน', 'การวางแผนการเงิน', 'เงินฉุกเฉิน', 'ลดค่าใช้จ่าย', 'ลดหย่อนภาษี', 'วินัยทางการเงิน', 'คำแนะนำการลงทุน', 'หนี้สิน', 'หุ้นกู้'])

In [5]:
for key, value in extracted_ids_keyword_dict.items():
    print(f'{key}: {len(value)}')

บำนาญ: 10000
หุ้นปันผล: 10000
หุ้นต่างประเทศ: 7240
อสังหา: 10000
หุ้นเน้นปันผล : 1792
กองทุน: 10000
สินทรัพย์: 10000
วางแผนชีวิต: 10000
วางแผนครอบครัว: 1940
แผนดูแลพ่อแม่: 0
Work-Life Balance: 620
เก็บเงินแต่งงาน: 4262
ให้เงินภรรยา: 886
ส่งเงินให้ที่บ้าน: 2950
ดูแลพ่อแม่: 10000
เสียสละเพื่อครอบครัว: 606
ทดแทนบุญคุณ: 4428
พ่อแม่ป่วย: 4488
ประกัน: 10000
ข้อมูลสุขภาพประกัน: 575
เดินตามรอย: 5707
heart rate: 3138
ข้อมูลสุขภาพ: 4607
Passive Income: 5058
กระแสเงินสด: 9230
เป้าหมายการเงิน: 1014
รายได้เสริม: 10000
อาชีพเสริม: 10000
รายได้2ทาง: 909
อิสรภาพทางการเงิน: 1355
รายได้หลายทาง: 2276
แยกกระเป๋า: 4970
กลยุทธ์การลงทุน: 5682
แนวทางการลงทุน: 3004
กลยุทธ์การเงิน: 375
Early retire: 703
เกษียณก่อนวัย: 284
เก็บเงินไว้ใช้ตอนเกษียณ: 109
เงินเก็บฉุกเฉิน: 1383
การวางแผนการเงิน: 3640
เงินฉุกเฉิน: 7199
ลดค่าใช้จ่าย: 10000
ลดหย่อนภาษี: 10000
วินัยทางการเงิน: 3349
คำแนะนำการลงทุน: 2080
หนี้สิน: 10000
หุ้นกู้: 10000


# Function to Scraping the Comment by sending topic_id

In [ ]:
def fetch_comments(key, comment_ids):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }

    all_comments = {}

    for comment_id in comment_ids:
        params = {
            'tid': comment_id,
            "type": "3"
        }

        try:
            r = requests.get("http://pantip.com/forum/topic/render_comments", params=params, headers=headers)
            r.raise_for_status()

            if r.status_code == 200:
                try:
                    comment_data = r.json()
                    all_comments[comment_id] = comment_data
                except json.JSONDecodeError as json_error:
                    print(f"JSON decoding error for {comment_id}: {json_error}")
            else:
                print(f"Unexpected status code {r.status_code} for {comment_id}")

        except (RequestException, HTTPError, ConnectionError, Timeout) as e:
            print(f"Request error for {comment_id}: {e}")
    
    return key, all_comments

def get_comment_ids(extracted_ids_keyword_dict, max_workers=10):
    comment_dict = {}
    # Limit the number of concurrent requests, this will be assigned to each thread 
    batch_size = 600
    
    # Use ThreadPoolExecutor to fetch comments concurrently
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        
        for key, value in extracted_ids_keyword_dict.items():
            if isinstance(value, list):
                # Create batches of up to 600 IDs, this will be divided among threads (20 threads = 12,000 IDs)
                batches = [value[i:i+batch_size] for i in range(0, len(value), batch_size)]
                # Submit each batch to the executor
                for batch in batches:
                    futures.append(executor.submit(fetch_comments, key, batch))
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            key, result = future.result()
            if result:
                if key not in comment_dict:
                    comment_dict[key] = []
                comment_dict[key].append(result)
            
            time.sleep(0.5)  # Add a small delay between requests
    
    # Merge results for each key into a single dictionary, for final output
    for key in comment_dict:
        merged_result = {}
        # Merge all results for each key from every batch
        for batch_result in comment_dict[key]:
            merged_result.update(batch_result)
        comment_dict[key] = merged_result
    
    return comment_dict

# Batch Crawl

In [ ]:
#seprarate the dictionary into 3, where key is divided into 3
def divide_dict(dictionary):
    dict1 = {}
    dict2 = {}
    dict3 = {}
    
    for i, key in enumerate(dictionary.keys()):
        if i % 3 == 0:
            dict1[key] = dictionary[key]
        elif i % 3 == 1:
            dict2[key] = dictionary[key]
        else:
            dict3[key] = dictionary[key]
    
    return dict1, dict2, dict3

In [ ]:
#Create 3 separate dictionaries for each thread
filtered_dict1, filtered_dict2, filtered_dict3 = divide_dict(filtered_dict)

In [ ]:
#Investigate a keys to prevent duplication
filtered_dict1.keys()

In [ ]:
filtered_dict2.keys()

In [ ]:
filtered_dict3.keys()

## Crawling the data

In [ ]:
result1 = get_comment_ids(filtered_dict1)

In [ ]:
result2 = get_comment_ids(filtered_dict2)

In [ ]:
result3 = get_comment_ids(filtered_dict3)

# Create a DataFrame from the results

In [ ]:
comment_df = pd.DataFrame.from_dict(result3, orient='index')

In [ ]:
#limit column to show only show 5 columns
pd.set_option('display.max_columns', 10)

comment_df.head(15)